In [1]:
import nltk
nltk.download('all')

[nltk_data] Downloading collection 'all'
[nltk_data]    | 
[nltk_data]    | Downloading package abc to /root/nltk_data...
[nltk_data]    |   Unzipping corpora/abc.zip.
[nltk_data]    | Downloading package alpino to /root/nltk_data...
[nltk_data]    |   Unzipping corpora/alpino.zip.
[nltk_data]    | Downloading package averaged_perceptron_tagger to
[nltk_data]    |     /root/nltk_data...
[nltk_data]    |   Unzipping taggers/averaged_perceptron_tagger.zip.
[nltk_data]    | Downloading package averaged_perceptron_tagger_ru to
[nltk_data]    |     /root/nltk_data...
[nltk_data]    |   Unzipping
[nltk_data]    |       taggers/averaged_perceptron_tagger_ru.zip.
[nltk_data]    | Downloading package basque_grammars to
[nltk_data]    |     /root/nltk_data...
[nltk_data]    |   Unzipping grammars/basque_grammars.zip.
[nltk_data]    | Downloading package bcp47 to /root/nltk_data...
[nltk_data]    | Downloading package biocreative_ppi to
[nltk_data]    |     /root/nltk_data...
[nltk_data]    |   U

True

In [2]:
import pandas as pd
import numpy as np
from nltk.tokenize import word_tokenize
from nltk import pos_tag
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
from sklearn.preprocessing import LabelEncoder
from collections import defaultdict
from nltk.corpus import wordnet as wn
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn import model_selection, naive_bayes, svm
from sklearn.metrics import accuracy_score



In [54]:
np.random.seed(500)

In [11]:
tag_map = defaultdict(lambda : wn.NOUN)
tag_map['J'] = wn.ADJ
tag_map['V'] = wn.VERB
tag_map['R'] = wn.ADV


In [25]:
!pip install joblib

In [26]:
from sklearn.naive_bayes import MultinomialNB
import joblib

In [55]:
train = pd.read_csv(r"/train.csv",encoding='latin-1')

In [56]:
train

,category,text
0,tech,tv future in the hands of viewers with home th...
1,business,worldcom boss left books alone former worldc...
2,sport,tigers wary of farrell gamble leicester say ...
3,sport,yeading face newcastle in fa cup premiership s...
4,entertainment,ocean s twelve raids box office ocean s twelve...
...,...,...
2220,business,cars pull down us retail figures us retail sal...
2221,politics,kilroy unveils immigration policy ex-chatshow ...
2222,entertainment,rem announce new glasgow concert us band rem h...
2223,politics,how political squabbles snowball it s become c...


In [57]:
print(tag_map)
word_Lemmatized = WordNetLemmatizer()
train['text']

defaultdict(<function <lambda> at 0x7a06f5a3e680>, {'J': 'a', 'V': 'v', 'R': 'r', 'N': 'n', 'M': 'n', 'D': 'n', 'I': 'n', 'C': 'n', 'S': 'n', 'P': 'n', '$': 'n', 'U': 'n', 'E': 'n', 'F': 'n', 'W': 'n', 'T': 'n', "'": 'n'})


0       tv future in the hands of viewers with home th...
1       worldcom boss  left books alone  former worldc...
2       tigers wary of farrell  gamble  leicester say ...
3       yeading face newcastle in fa cup premiership s...
4       ocean s twelve raids box office ocean s twelve...
                              ...                        
2220    cars pull down us retail figures us retail sal...
2221    kilroy unveils immigration policy ex-chatshow ...
2222    rem announce new glasgow concert us band rem h...
2223    how political squabbles snowball it s become c...
2224    souness delight at euro progress boss graeme s...
Name: text, Length: 2225, dtype: object

In [58]:
train['text'].dropna(inplace=True)

In [61]:
train['text']= [word_tokenize(entry) for entry in train['text']]

In [62]:
train.head

<bound method NDFrame.head of            category                                               text
0              tech  [tv, future, in, the, hands, of, viewers, with...
1          business  [worldcom, boss, left, books, alone, former, w...
2             sport  [tigers, wary, of, farrell, gamble, leicester,...
3             sport  [yeading, face, newcastle, in, fa, cup, premie...
4     entertainment  [ocean, s, twelve, raids, box, office, ocean, ...
...             ...                                                ...
2220       business  [cars, pull, down, us, retail, figures, us, re...
2221       politics  [kilroy, unveils, immigration, policy, ex-chat...
2222  entertainment  [rem, announce, new, glasgow, concert, us, ban...
2223       politics  [how, political, squabbles, snowball, it, s, b...
2224          sport  [souness, delight, at, euro, progress, boss, g...

[2225 rows x 2 columns]>

In [63]:
for index, entry in enumerate(train['text']):
    # print(index, entry)
    # print("--------------------------------")
    Final_words = []
    for word, tag in pos_tag(entry):
        # print(word, tag)
        if word not in stopwords.words('english') and word.isalpha():
            word_Final = word_Lemmatized.lemmatize(word,pos = tag_map[tag[0]])
            Final_words.append(word_Final)
    train.loc[index,'text_final'] = str(Final_words)

In [64]:
train

,category,text,text_final
0,tech,"[tv, future, in, the, hands, of, viewers, with...","['tv', 'future', 'hand', 'viewer', 'home', 'th..."
1,business,"[worldcom, boss, left, books, alone, former, w...","['worldcom', 'bos', 'leave', 'book', 'alone', ..."
2,sport,"[tigers, wary, of, farrell, gamble, leicester,...","['tiger', 'wary', 'farrell', 'gamble', 'leices..."
3,sport,"[yeading, face, newcastle, in, fa, cup, premie...","['yeading', 'face', 'newcastle', 'fa', 'cup', ..."
4,entertainment,"[ocean, s, twelve, raids, box, office, ocean, ...","['ocean', 'twelve', 'raid', 'box', 'office', '..."
...,...,...,...
2220,business,"[cars, pull, down, us, retail, figures, us, re...","['car', 'pull', 'u', 'retail', 'figure', 'u', ..."
2221,politics,"[kilroy, unveils, immigration, policy, ex-chat...","['kilroy', 'unveils', 'immigration', 'policy',..."
2222,entertainment,"[rem, announce, new, glasgow, concert, us, ban...","['rem', 'announce', 'new', 'glasgow', 'concert..."
2223,politics,"[how, political, squabbles, snowball, it, s, b...","['political', 'squabble', 'snowball', 'become'..."


In [65]:
train_X, test_X, train_Y, test_Y = model_selection.train_test_split(train['text_final'],train['category'],test_size=0.3)

In [66]:
train_X

714     ['amex', 'share', 'news', 'share', 'american',...
2106    ['u', 'trade', 'deficit', 'widen', 'sharply', ...
92      ['goldsmith', 'leant', 'attorney', 'general', ...
907     ['butler', 'launch', 'attack', 'blair', 'forme...
1180    ['reliance', 'unit', 'lose', 'anil', 'ambani',...
                              ...                        
845     ['brazil', 'jobless', 'rate', 'hit', 'new', 'l...
584     ['fa', 'decide', 'punish', 'mourinho', 'footba...
287     ['peer', 'debate', 'crown', 'succession', 'law...
273     ['u', 'critic', 'laud', 'comedy', 'sideways', ...
951     ['csa', 'chief', 'quit', 'still', 'job', 'head...
Name: text_final, Length: 1557, dtype: object

In [67]:
test_X

1252    ['blinx', 'sequel', 'purr', 'nicely', 'origina...
1162    ['moyes', 'beattie', 'dismissal', 'everton', '...
395     ['holmes', 'hit', 'hamstring', 'injury', 'kell...
1396    ['bellamy', 'fin', 'row', 'newcastle', 'fin', ...
1132    ['profit', 'jump', 'china', 'top', 'bank', 'in...
                              ...                        
1894    ['eu', 'software', 'patent', 'law', 'face', 'a...
23      ['cole', 'face', 'lengthy', 'injury', 'aston',...
733     ['comeback', 'show', 'friend', 'star', 'friend...
2056    ['student', 'inequality', 'expose', 'teenager'...
1448    ['film', 'row', 'pirate', 'cannibal', 'plan', ...
Name: text_final, Length: 668, dtype: object

In [68]:
train_Y

714          business
2106         business
92           politics
907          politics
1180         business
            ...      
845          business
584             sport
287          politics
273     entertainment
951          politics
Name: category, Length: 1557, dtype: object

In [69]:
test_Y

1252             tech
1162            sport
395             sport
1396            sport
1132         business
            ...      
1894             tech
23              sport
733     entertainment
2056         politics
1448    entertainment
Name: category, Length: 668, dtype: object

In [70]:
Encoder = LabelEncoder()
train_Y = Encoder.fit_transform(train_Y)
test_Y = Encoder.fit_transform(test_Y)

In [72]:
train_Y

array([0, 0, 2, ..., 2, 1, 2])

In [73]:
test_Y

array([4, 3, 3, 3, 0, 2, 3, 4, 1, 2, 4, 0, 3, 1, 0, 0, 1, 3, 1, 2, 4, 1,
       0, 1, 2, 4, 2, 4, 4, 0, 2, 2, 0, 2, 2, 0, 1, 0, 4, 4, 2, 0, 0, 4,
       2, 2, 0, 0, 1, 3, 0, 4, 4, 0, 0, 0, 0, 1, 3, 3, 0, 4, 4, 4, 3, 1,
       2, 1, 2, 3, 3, 0, 0, 4, 3, 1, 0, 4, 1, 1, 2, 1, 0, 3, 4, 0, 0, 4,
       1, 1, 4, 0, 4, 1, 4, 3, 4, 3, 1, 4, 2, 4, 0, 2, 0, 4, 0, 3, 3, 4,
       3, 1, 0, 2, 0, 0, 3, 0, 1, 4, 2, 2, 0, 2, 1, 4, 3, 1, 0, 4, 0, 2,
       2, 0, 0, 4, 3, 4, 4, 1, 4, 3, 1, 1, 3, 3, 2, 2, 1, 4, 0, 4, 0, 2,
       3, 2, 0, 3, 0, 0, 4, 2, 2, 2, 2, 0, 2, 1, 3, 4, 2, 4, 0, 0, 0, 2,
       1, 2, 1, 1, 0, 4, 0, 1, 3, 4, 1, 3, 1, 3, 4, 4, 3, 0, 3, 3, 3, 2,
       1, 0, 0, 0, 2, 4, 0, 2, 3, 2, 1, 4, 3, 1, 4, 4, 4, 0, 4, 2, 2, 3,
       2, 2, 4, 3, 4, 3, 3, 3, 1, 4, 2, 2, 0, 1, 3, 2, 4, 2, 4, 3, 4, 3,
       0, 0, 4, 2, 2, 2, 1, 3, 1, 3, 3, 0, 0, 0, 2, 4, 4, 3, 0, 4, 4, 1,
       1, 3, 3, 1, 2, 3, 4, 4, 2, 0, 1, 1, 4, 4, 3, 0, 3, 2, 0, 2, 4, 1,
       0, 3, 3, 0, 0, 3, 1, 1, 2, 4, 2, 2, 4, 3, 2,

In [71]:
Tfidf_vect = TfidfVectorizer(max_features=5000)
Tfidf_vect.fit(train['text_final'])

TfidfVectorizer(max_features=5000)

In [74]:
train_X_Tfidf = Tfidf_vect.transform(train_X)
test_X_Tfidf = Tfidf_vect.transform(test_X)

In [83]:
train_X_Tfidf

<1557x5000 sparse matrix of type '<class 'numpy.float64'>'
	with 189717 stored elements in Compressed Sparse Row format>

In [84]:
test_X_Tfidf

<668x5000 sparse matrix of type '<class 'numpy.float64'>'
	with 81535 stored elements in Compressed Sparse Row format>

In [75]:
NB = naive_bayes.MultinomialNB()
NB.fit(train_X_Tfidf,train_Y)

MultinomialNB()

In [76]:
pred_NB = NB.predict(test_X_Tfidf)


In [77]:
print("Naive Bayes Accuracy Score -> ",accuracy_score(pred_NB, test_Y)*100)

Naive Bayes Accuracy Score ->  97.30538922155688


In [78]:
SVM = svm.SVC(C = 1.0, kernel = 'linear', degree = 3, gamma = 'auto')

In [79]:
SVM.fit(train_X_Tfidf, train_Y)

SVC(gamma='auto', kernel='linear')

In [80]:
pred_SVM = SVM.predict(test_X_Tfidf)

In [81]:
print("SVM Accuracy Score -> ",accuracy_score(pred_SVM, test_Y)*100)

SVM Accuracy Score ->  98.35329341317365












**For testing**









For testing the above trained models, preprocess the raw data, specifically the text column to be tested. For this use the below:




In [ ]:
# for index, entry in enumerate((dataframe_name)['text']):
#     # print(index, entry)
#     # print("--------------------------------")
#     Final_words = []
#     for word, tag in pos_tag(entry):
#         # print(word, tag)
#         if word not in stopwords.words('english') and word.isalpha():
#             word_Final = word_Lemmatized.lemmatize(word,pos = tag_map[tag[0]])
#             Final_words.append(word_Final)
#     (dataframe_name).loc[index,'text_final'] = str(Final_words)


You can directly use the trained model here without saving it.

In [ ]:
# test = (dataframe_name).['text_final']
# test_Tfidf = Tfidf_vect.transform(test)
# prediction_NB = NB.predict(test_Tfidf)

# prediction_NB

In [ ]:
#(dataframe_name)['Prediction_NB'] = prediction_NB

In [ ]:
# prediction_SVM = SVM.predict(test_Tfidf)
# prediction_SVM

In [ ]:
#(dataframe_name)['Prediction_SVM'] = prediction_SVM

Otherwise to save the trained model and load the saved model, you can use the following code :

In [ ]:
# NB.save('NB.pkl')
# model_NB = joblib.loac('NB.pkl')

# prediction_NB = model_NB.predict(test_Tfidf)
# prediction_NB

In [ ]:
# SVM.save('SVM.pkl')
# model_SVM = joblib.load('SVM.pkl')

# prediction_SVM = model_SVM.predict(test_Tfidf)
# prediction_SVM